In [ ]:
import sys
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as cmo
import cartopy.crs as ccrs

sys.path.append("..")

from plotfunctions import *
import warnings
warnings.filterwarnings("ignore")

np.seterr(all='ignore')
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
mpl.rcParams['figure.subplot.bottom'] = .15
mpl.rcParams['figure.subplot.top']    = .99
mpl.rcParams['figure.subplot.left']   = .01
mpl.rcParams['figure.subplot.right']  = .99
mpl.rcParams['figure.subplot.wspace'] = .02
mpl.rcParams['figure.subplot.hspace'] = .01

In [ ]:
"""Boundaries MITgcm"""
y1,y2 = {},{}
x1,x2 = {},{}
x1['CrossDots']  = 245
x2['CrossDots']  = 251
y1['CrossDots']  = -75.45
y2['CrossDots']  = -74.05
x1['Thwaites_e'] = 251
x2['Thwaites_e'] = 257
y1['Thwaites_e'] = -75.45
y2['Thwaites_e'] = -74.05
x1['PineIsland'] = 257
x2['PineIsland'] = 262
y1['PineIsland'] = -75.45
y2['PineIsland'] = -74.05
x1['Getz']       = 225
x2['Getz']       = 245.3
y1['Getz']       = -75.45
y2['Getz']       = -73.2

"""Locations for text"""
plon,plat = {},{}
plon['CrossDots']  = 250.5
plat['CrossDots']  = -74.4
plon['Thwaites_e'] = 253.5
plat['Thwaites_e'] = -74.6
plon['PineIsland'] = 257
plat['PineIsland'] = -74.8
plon['Getz']       = 231
plat['Getz']       = -74

In [ ]:
#Parameters MITgcm
timep= slice("1989-1-1","2018-12-31")

#Parameters Layer
forc_w = {}
forc_c = {}
forc_w['PineIsland'] = 'tanh2_Tdeep1.2_ztcl-400'
forc_c['PineIsland'] = 'tanh2_Tdeep1.2_ztcl-600'
forc_w['Thwaites_e'] = 'tanh2_Tdeep1.2_ztcl-400'
forc_c['Thwaites_e'] = 'tanh2_Tdeep1.2_ztcl-600'
forc_w['CrossDots']  = 'tanh2_Tdeep0.7_ztcl-400'
forc_c['CrossDots']  = 'tanh2_Tdeep0.7_ztcl-700'
forc_w['Getz']       = 'tanh2_Tdeep0.7_ztcl-400'
forc_c['Getz']       = 'tanh2_Tdeep0.7_ztcl-700'

spinup = {}
spinup['PineIsland'] = 180
spinup['Thwaites_e'] = 60
spinup['CrossDots']  = 180
spinup['Getz']       = 10

date = '2021_02_25/'


In [ ]:
panelheight = .2
step = .125
figheight = 11

proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=245-360)

def addaxis(n,title):
    pos = 1-panelheight-n*step
    ax = fig.add_axes([.01,pos,.98,panelheight],projection=proj)
    ds = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3000,4500),y=slice(7700,9500))
    ds = add_lonlat(ds)
    mask = xr.where(ds.mask==0,4,ds.mask)
    ax.contour(ds.lon,ds.lat,mask,[2.5],c='k',linewidths=.2,transform=ccrs.PlateCarree())

    ds = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3300,4000),y=slice(6000,7700))
    ds = add_lonlat(ds)
    mask = xr.where(ds.mask==0,4,ds.mask)
    ax.contour(ds.lon,ds.lat,mask,[2.5],c='k',linewidths=.2,transform=ccrs.PlateCarree())
    ax.set_extent([227,259,-75.2,-73.1],crs=ccrs.PlateCarree())
    ax.set_title(title,y=.5,loc='left')
    ax.axis('off')
    return ax

def printvals(ax,geom,melt):
    ax.text(plon[geom],plat[geom],f'{np.nanmean(melt):.1f}',transform=ccrs.PlateCarree(),c='k',ha='center')


In [ ]:
fig = plt.figure(figsize=(8,figheight))

"""Geometry"""
ax = addaxis(0,'a) Geometry')

ds = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(3000,4500),y=slice(7700,9500))
ds = add_lonlat(ds)
draft = xr.where(ds.mask==3,(ds.surface-ds.thickness).astype('float64'),np.nan)
draft = xr.where((ds.lon>-105).all() and (ds.lat>-73.7).all(),np.nan,draft)
IM = plt.pcolormesh(ds.lon,ds.lat,draft,vmin=-1000,vmax=0,cmap=plt.get_cmap('cmo.rain_r'),transform=ccrs.PlateCarree(),shading='auto')
ds = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(3300,4000),y=slice(6000,7700))
ds = add_lonlat(ds)
draft = xr.where(ds.mask==3,(ds.surface-ds.thickness).astype('float64'),np.nan)
draft = xr.where((ds.lon>-105).all() and (ds.lat>-73.7).all(),np.nan,draft)
IM = plt.pcolormesh(ds.lon,ds.lat,draft,vmin=-1000,vmax=0,cmap=plt.get_cmap('cmo.rain_r'),transform=ccrs.PlateCarree(),shading='auto')

'''Colorbar'''
cax = fig.add_axes([.1,.05,.35,.015])
cbar = plt.colorbar(IM, cax=cax,extend='min',orientation='horizontal')
cbar.set_label('Draft depth [m]', labelpad=0)

"""M+"""
ax = addaxis(1,'b) M+')
#Plot melt here

"""PICO"""
ax = addaxis(2,'c) PICO')
#Plot melt here

"""Plume"""
ax = addaxis(3,'d) Plume')
#Plot melt here

"""PICOP"""
ax = addaxis(4,'e) PICOP')
#Plot melt here

"""Layer"""
ax = addaxis(5,'f) Layer')
for geom in ['PineIsland','Thwaites_e','CrossDots','Getz']:
    ds = xr.open_dataset(f'../../results/Layer/{date}{geom}_{forc_w[geom]}_{spinup[geom]:.3f}.nc')
    melt_w = np.where(ds.mask==3,ds.melt,np.nan)
    ds = xr.open_dataset(f'../../results/Layer/{date}{geom}_{forc_c[geom]}_{spinup[geom]:.3f}.nc')
    ds = add_lonlat(ds)
    melt_c = np.where(ds.mask==3,ds.melt,np.nan)    
    
    melt = .5*(melt_w+melt_c)
    printvals(ax,geom,melt)
    melt = np.where(melt<1,1,melt)
    IM = plotmelt(ax,ds.lon,ds.lat,melt)    

"""MITgcm"""
ax = addaxis(6,'g) MITgcm')
for geom in ['PineIsland','Thwaites_e','CrossDots','Getz']:
    ds = xr.open_dataset('../../data/paulholland/melt.nc')
    ds = ds.sel(LONGITUDE=slice(x1[geom],x2[geom]),LATITUDE=slice(y1[geom],y2[geom]),TIME=timep)
    ds = ds.mean(dim='TIME')
    lon = ds.LONGITUDE
    lat = ds.LATITUDE-.05
    melt = xr.where(ds.melt==0,np.nan,ds.melt)
    
    printvals(ax,geom,melt)
    IM = plotmelt(ax,lon,lat,melt)

'''Colorbar'''
cax = fig.add_axes([.5,.05,.35,.015])
cbar = plt.colorbar(IM, cax=cax,extend='both',orientation='horizontal')
cbar.set_ticks([1,10,100])
cbar.set_ticklabels([1,10,100])
cbar.set_label('Melt rate [m/yr]', labelpad=0)

"""Save figure"""
plt.savefig(f"../../figures/Amundsen_average.png",dpi=300)
plt.show()